In [ ]:
!pip install SPARQLWrapper

In [ ]:
!pip3 install https://github.com/explosion/spacy-models/releases/download/en_core_web_md-2.2.0/en_core_web_md-2.2.0.tar.gz

You should provide a file named 'Attribute dictionary.txt', helping convert unencodable characters to encodable characters

In [10]:
from SPARQLWrapper import SPARQLWrapper
import en_core_web_md
import spacy
import numpy as np
from tqdm import tqdm
import pickle

In [5]:
dic = {}
with open('Attribute dictionary.txt','r') as f:
  for line in f:
    k,v= line.strip().split(',')
    dic[k]=v

In [6]:
def get_data(query,res_format):
    sparql = SPARQLWrapper("https://dbpedia.org/sparql/")
    sparql.setQuery(query)
    sparql.setReturnFormat(res_format)
    try:
        ret = sparql.query().convert()
        return ret
    except:
        #deal_with_the_exception()
        #print("there's somthing wrong!\n")
        return 0
key = ['http://dbpedia.org/ontology/','http://dbpedia.org/resource/','http://dbpedia.org/property/']
value_type_skip = ['typed-literal'] 

In [7]:
class consider_relationship_and_entity:
    '''
    consider the relationships and entities at the same time 
    '''
    def __init__(self,center_word = None,escape = True,num=10,res_format = 'json',weight_degree  =0.5,weight_relationship = 0.5):
        self.center_word = center_word
        self.escape = escape #是否判断key和跳过value_type_skip
        self.query = "" #得到查询语句
        self.top_k = num  #最终结果展示的个数
        self.format = res_format #查询语句返回的形式
        #self.nlp = en_core_web_sm.load()
        self.nlp = en_core_web_md.load()
        self.weight_degree = weight_degree
        self.weight_relationship = weight_relationship
    
    def _new_centerword(self,center_word):
        self.center_word  =center_word
    
    def _get_centerword(self,center_word):
        return self.center_word
    
    def _get_query(self,center_word,is_for_expand):
        center_word = center_word.replace(' ','_')
        if not is_for_expand: #is_for_expand == False
            self.query =[ 
                f'''
                prefix dbo:<http://dbpedia.org/ontology/> 
                prefix dbr:<http://dbpedia.org/resource/> 
                SELECT ?property ?value 
                Where {{ 
                dbr:{self.center_word} ?property  ?value 
                }}
                '''
                ,
                f'''
                prefix dbo:<http://dbpedia.org/ontology/> 
                prefix dbr:<http://dbpedia.org/resource/> 
                SELECT  ?value ?property
                Where {{ 
                 ?value ?property dbr:{self.center_word}
                }}
                '''
            ]
                
        else:
            self.query = [f'''
        prefix dbo:<http://dbpedia.org/ontology/> 
        prefix dbr:<http://dbpedia.org/resource/> 
        SELECT COUNT(?label ) as ?num  
        Where{{ 
        dbr:{center_word} ?label ?value
        }}
        '''
           ,
            f'''
        prefix dbo:<http://dbpedia.org/ontology/> 
        prefix dbr:<http://dbpedia.org/resource/> 
        SELECT COUNT(?label ) as ?num  
        Where{{ 
        ?value ?label  dbr:{center_word}
        }}
        ''']
    
    def _clean_data(self,data,as_subject=True):
        #这里不用 fliter而用遍历是因为返回值都为uri，还是要遍历取出string方便画图
        #只保留以key开头的边
        #且去除节点为value_type_skip的值的节点
        #最终保留在w中
        '''
        形式：
        [,,,]
        只保存节点，边的属性可以在查出top_10后再添加
        '''
        w = []
        for i in data["results"]["bindings"]:
            if i['property']['value'].startswith(key[0]) or i['property']['value'].startswith(key[1]) or i['property']['value'].startswith(key[2]):
                if i['value']['type'] in value_type_skip:
                    continue
                if i['value']['type'] == 'uri':
                    v = i['value']['value'].split('/')[-1]
                elif  i['value']['type'] == 'literal' and i['value']['xml:lang'] == "en" and i['value']['value']!= "": #注意有为空的情况
                    v = i['value']['value']
                else:
                    continue
                l = i['property']['value'].split('/')[-1]#查看过其返回类型都是uri
                if l.startswith('wiki'): #dbo中也有wiki
                    continue
                if len(w)>0 and v in w: #若此时节点已经记录了
                    continue
                else:
                    if v == self.center_word:
                        print(i)
                        continue
                    w.append(v)

                
        return w
    
    def _get_the_relationship(self,subject_,object_):
        'here return relationship as turple'
        relationships = []
        subject_ = subject_.replace(' ','_')
        object_ = object_.replace(' ','_')
        query = f'''
                prefix dbo:<http://dbpedia.org/ontology/> 
                prefix dbr:<http://dbpedia.org/resource/> 
                SELECT ?property
                Where {{ 
                dbr:{subject_} ?property  dbr:{object_} 
                }}
                '''
        
        data = get_data(query,self.format)
        if data == 0: #they don't have realtionship from this direction
            return None
        for i in data["results"]["bindings"]:
            l = i['property']['value'].split('/')[-1]#查看过其返回类型都是uri
            if l.startswith('wiki'): #dbo中也有wiki
                continue
            else:
                relationships.append(l)
        
        return relationships
    
    def _get_data(self,exist_pickle = False):
        if exist_pickle:
          with open(f'{self.center_word}.pckl','rb') as f:
            w = pickle.load(f)
          w = self._get_the_topk_by_RD(w,self.top_k)
          print("the top 10:")
          print(w) #show the top-10 degree words
        
        
          return w 

        print("=== run the get data process ===== ")
        self._get_query(self.center_word,False)
        data = get_data(self.query[0],self.format) # 作主语
        if data == 0:
            print(data)
            w = [] 
        else:
            print(len(data))
            print(f'[!] as the subject: num  = {len(data["results"]["bindings"])}')
        #print(len(data))
            if self.escape:
                w = self._clean_data(data)
            else:
                w = data["results"]["bindings"] #？感觉这个else没有写对，就算不跳过，也应该从uri中获得strig
            print(f'[!] as the subject after cleanning: num  = {len(w)}')
        
        data = get_data(self.query[1],self.format) # 作宾语
        if data == 0:
            print(f'[!] as the object: num  = {data}')
            w_ = []
                
        else:
            print(f'[!] as the object: num  = {len(data["results"]["bindings"])}')
            if self.escape:
                w_ = self._clean_data(data)
            else:
                w_ = data["results"]["bingdings"]#？感觉这个else没有写对，就算不跳过，也应该从uri中获得strig
            print(f'[!] as the object after cleanning: num  = {len(w_)}')
        
        w = w + w_

        w = [i.replace('_',' ') for i in w]  #去重
        print('w\n',w)
        w = list(set(w)) #去重
        print('w_set\n',w)
        with open(f'{self.center_word}.pckl','wb') as f:
          pickle.dump(w,f)
        
        # call _get_the_top10_by_degree()
        w = self._get_the_topk_by_RD(w,self.top_k)
        #print("the top 10:")
        #print(w) #show the top-10 degree words
        
        
        
        return w 
    
    def _get_the_degree(self,words):
        w = []
        total_num = 0 #用于做度的归一化
        for i in tqdm(range(len(words))):#对每一个结果查询其周围的度,出度（作主语）和入度（作宾语）
            self._get_query(words[i],True)
            res = get_data(self.query[0],self.format)#作主语
            if res != 0:
                num =int(res["results"]["bindings"][0]["num"]["value"])
            else:
                num = 0
                
            res = get_data(self.query[1],self.format)#作宾语
            if res != 0:
                num +=int(res["results"]["bindings"][0]["num"]["value"])
            else:
                num += 0
            total_num += num
            w.append({'value':words[i],'degree':num})
        
        

        return w
    
    def _get_the_norm_relationship(self,w):
        #获得关系
        nodes = []
        '''
        {
          value:
          direction:
          degree:
          label:
        }
        '''

        relationship_matrix = []
        map_value_relationship = {}
        for i in w:
            node = i['value']
            relationship =self._get_the_relationship(subject_=self.center_word,object_=node)
            i['direction'] = 'out'
            if relationship == None:
                relationship = self._get_the_relationship(subject_=node,object_=self.center_word)
                i['direction'] = 'in'
            if relationship == None:
              print('relationship == None',"  ",i)
              continue
            
            '''
            处理1：不同的属性，节点一样，也看作独立的属性点
            '''
            same = [] #避免同一节点的相同属性加多次
            string_r = ''
            for r in relationship:
              relationship = r
              string_r = string_r +'/' +r
              if self.nlp(r)[0].is_oov:
                label = dic[str(r)]
              else:
                label = r
              if len(same) == 0:
                same.append(label)
              elif label in same:
                continue
              
              nodes.append({
                  'degree': i['degree'],
                  'direction': i['direction'],
                  'label': label,
                  'relationship': relationship,
                  'value': i['value']
              })
              relationship_matrix.append(label)
            map_value_relationship[i['value']] = string_r
            

        print('map_value_relationship\n',map_value_relationship)
        '''

        魔改：先把去重之后，再计算相似度，看一下情况

        '''
        relationship_matrix = list(set(relationship_matrix))





        print("relationship_matrix",relationship_matrix)
        relationship_vector = [self.nlp(r).vector if not self.nlp(r)[0].is_oov else self.nlp(dic[str(r)]).vector for r in relationship_matrix] #不确定是不是这样堆叠起来



        #print("relationship_vector",relationship_vector)
        trans_relationship_vector = np.transpose(relationship_vector)
        similarity = np.dot(relationship_vector,trans_relationship_vector) 
        #print("similarity",similarity)
        norm =  np.array([np.linalg.norm(r) for r in relationship_vector])# 不加np.array下一步会报错
        normalization = [r*norm for r in norm]
        #print("norm",norm)
        #print("normalization",normalization)
        similarity_norm = similarity / normalization #得到【0~1】的相似度
        #print("similarity_norm",similarity_norm)
        
        return similarity_norm,nodes,relationship_matrix
        
        
        
        
        
    
    def _get_the_topk_by_RD(self,words,top_k = 10):
        
        w = self._get_the_degree(words) 
        #print('get_the_degree_w\n',w)
        norm_similarity,w,relationship_matrix = self._get_the_norm_relationship(w)
        print('get_the_relation_w\n',w)
        #??注意当同一个entity有多个relationship时如何处理,【目前方法是直接相加】
        '''
        w =[
        {
        'direction':
        'value':
        'relationship
        'label':
        'degree':
        }
        .....
        ]
        
        '''
        num_total =  np.size(norm_similarity,axis=1) #得到列数，即一行有多少
        sum_norm_similarity = np.sum(norm_similarity,axis = 1) #按行求和
        print("norm_similarity\n",norm_similarity)
        
        #打印一下相似度和与关系
        print(len(relationship_matrix))
        print(len(sum_norm_similarity))
        out =[ {'r':i,'sum_sim':j} for i,j in zip(relationship_matrix,sum_norm_similarity)]
        out = sorted(out,key=lambda k:k['sum_sim'],reverse=True)

        print(out)

        cluster = {}
        print(w)
        for i in w:
          if cluster.get(i['label']) == None:
            cluster[i['label']] = []

          cluster[i['label']].append(
                {
                    'value':i['value'],
                    'degree':i['degree']
                }
            )
        print('cluster\n',cluster)

        color_metric = [-1]*len(relationship_matrix)
        # =====对cluster先做合并
        th = 0.8
        for idx,j in enumerate(norm_similarity):
          if idx == len(norm_similarity)-1:
            continue
          else:
            s = j[idx+1:]
          for h,val in enumerate(s):
            if val >= th:
              print(f'{idx},{h}')
              if color_metric[idx] != -1: #此时节点被其他节点染色了
                color_metric[idx+1+h] = color_metric[idx]
              else:
                color_metric[idx+1+h] = idx
        print('relation\n',relationship_matrix)
        print('color_metric\n',color_metric)

        map_oldRelationship_new = {}

        to_remove = []
        for idx,val in enumerate(color_metric):
          if val != -1:#此时节点被其他节点染色了
            map_oldRelationship_new[relationship_matrix[idx]] = relationship_matrix[val]
            for j in cluster[relationship_matrix[idx]]:
              cluster[relationship_matrix[val]].append(j)
            to_remove.append(relationship_matrix[idx])
        
        for r in to_remove:
          cluster.pop(r)
        
        # 把label统一，方便后续计算
        for i in w:
          if i['label'] in to_remove:
            i['label'] = map_oldRelationship_new[i['label']]
        
        print('after_clustered\n',cluster)
        #取前top_k乘以衰减减系数
        for k,v in cluster.items():
          v=sorted(v,key=lambda k:k['degree'],reverse=True)
          cluster[k]=v
        print('after_sort\n',cluster)


        # =====按cluster做归一化
        norm_degree = {}
        for k,v in cluster.items():
          norm_degree[k] = {} #避免不同cluster但是相同value的计算后，对归一化结果产生影响
          total = 0
          idx = 1
          for i in v:
            total += i['degree']
          for i in v:
            norm_degree[k][i['value']] = i['degree']/total*(1/idx) #会被减成负的也好，但是如果在上方取top_k则，度的归一化的分母会出错
            idx += 1
        
        #=== method 2：ignore the cluster_size but relationship similrity

        for i in w:
          i['norm_degree'] = norm_degree[i['label']][i['value']]
        
        w = sorted(w,key=lambda k:k['norm_degree'],reverse=True)

        n = 0
        res = []
        va = []
        idx = 0
        while n < top_k:
          if len(res) == 0:
            res.append(w[idx])
            va.append(w[idx]['value'])
            idx += 1
          else:
            if w[idx]['value'] in va:
              idx += 1
            else:
              res.append(w[idx])
              va.append(w[idx]['value'])
              idx += 1
              n += 1
        print(va)
        print('n',n)

          


        

        return res

In [8]:
import json
def json_w(w,path):
  res = {}
  for each in w :
    res[each['value']] = []
    res[each['value']].append({'label':each['label'],'degree':each['degree'],'score':each['norm_degree']})
  with open(path+'.json','w') as f:
    json.dump(res,f)

In [11]:
method = consider_relationship_and_entity(center_word="James_Clerk_Maxwell",weight_degree=10)
w = method._get_data(exist_pickle=False)
print(w)


=== run the get data process ===== 
2
[!] as the subject: num  = 957
[!] as the subject after cleanning: num  = 49


  0%|          | 0/56 [00:00<?, ?it/s]

[!] as the object: num  = 1408
[!] as the object after cleanning: num  = 14
w
 ['James Clerk Maxwell', 'James Clerk Maxwell.png?width=300', 'Cambridge', 'Cambridge, England, United Kingdom', 'Edinburgh', 'Edinburgh, Scotland, United Kingdom', 'George Chrystal', 'Horace Lamb', 'John Henry Poynting', 'William Hopkins', 'University of Aberdeen', 'University of Cambridge', "King's College London", 'Marischal College', 'University of Edinburgh', 'Fellow of the Royal Society', 'Fellowship of the Royal Society of Edinburgh', 'Adams Prize', "Smith's Prize", 'Rumford Medal', 'Keith Prize', 'British', 'Physics and mathematics', 'Maxwell,+James+Clerk', 'Virtually all subsequent physics', 'Michael Faraday', 'Isaac Newton', 'Maxwell relations', 'Color triangle', "Maxwell's theorem", 'Generalized Maxwell model', "Maxwell's equations", 'Maxwell–Boltzmann distribution', 'Displacement current', 'Maxwell material', 'Maxwell coil', "Maxwell's demon", "Maxwell's wheel", 'Scottish', 'Katherine Clerk Maxwel

100%|██████████| 56/56 [02:00<00:00,  2.15s/it]


relationship == None    {'value': 'James Clerk Maxwell FRSE FRS (13 June 1831 – 5 November 1879) was a Scottish scientist in the field of mathematics and mathematical physics. His most notable achievement was to formulate the classical theory of electromagnetic radiation, bringing together for the first time electricity, magnetism, and light as different manifestations of the same phenomenon. Maxwell\'s equations for electromagnetism have been called the "second great unification in physics" where the first one had been realised by Isaac Newton. With the publication of "A Dynamical Theory of the Electromagnetic Field" in 1865, Maxwell demonstrated that electric and magnetic fields travel through space as waves moving at the speed of light. He proposed that light is an undulation in the same medium that is the cause of electric and magnetic phenomena. The unification of light and electrical phenomena led his prediction of the existence of radio waves. Maxwell is also regarded as a found

In [13]:
w

[{'degree': 95,
  'direction': 'out',
  'label': 'spouse',
  'norm_degree': 1.0,
  'relationship': 'spouse',
  'value': 'Katherine Clerk Maxwell'},
 {'degree': 465,
  'direction': 'out',
  'label': 'academic advisor',
  'norm_degree': 1.0,
  'relationship': 'academicAdvisors',
  'value': 'William Hopkins'},
 {'degree': 41328,
  'direction': 'out',
  'label': 'birth place',
  'norm_degree': 1.0,
  'relationship': 'birthPlace',
  'value': 'Edinburgh'},
 {'degree': 16983,
  'direction': 'out',
  'label': 'death place',
  'norm_degree': 1.0,
  'relationship': 'deathPlace',
  'value': 'Cambridge'},
 {'degree': 34851,
  'direction': 'out',
  'label': 'workplaces',
  'norm_degree': 0.8522485510967647,
  'relationship': 'workplaces',
  'value': 'University of Cambridge'},
 {'degree': 40226,
  'direction': 'out',
  'label': 'academic discipline',
  'norm_degree': 0.6264073376208792,
  'relationship': 'academicDiscipline',
  'value': 'Mathematics'},
 {'degree': 23991,
  'direction': 'out',
  'la